In [ ]:
#!pip install -U qiskit
#!pip install -U qiskit_nature
#!pip install -U qiskit_aer
#!pip install -U qiskit_algorithms
#!pip install -U qiskit_ibm_runtime
#!pip install -U qiskit-aer-gpu
#!pip freeze | grep qiskit

In [ ]:
### Qiskit Nature
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal


from qiskit_nature.second_q.mappers import BravyiKitaevMapper,BravyiKitaevSuperFastMapper,JordanWignerMapper,ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC, UCCSD
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, ExcitedStatesEigensolver, QEOM, EvaluationRule
from qiskit_nature.second_q.transformers import FreezeCoreTransformer


### Qiskit Runtime
from qiskit.primitives import BackendEstimator as Estimator
from qiskit import QuantumCircuit,transpile
from qiskit_aer import AerSimulator, Aer
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService


### Qiskit Algorithms
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, SLSQP,COBYLA,SPSA
from qiskit_algorithms import NumPyMinimumEigensolver


from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint

### Other
import numpy as np
import copy
import warnings
warnings.filterwarnings("ignore")
from decimal import Decimal
from lib.molecule import Molecule
from lib.grid import generate_molecule_grid


In [ ]:
maxdx = .05
dxs = np.linspace(-maxdx, maxdx, 30)

def compute_energy_and_grad(mol, displacement_index, shots, qubit_reduction, mapper_type, parameters=None, opt=SLSQP()):


    # Molecule displacement setup
    #############################################################################
    
    molecule = mol
    problem = molecule.get_pyscf_driver(unit=unit, basis=basis)
    e_nr = problem.nuclear_repulsion_energy
    main_op = problem.hamiltonian.second_q_op()
    
    atom_idx, coord_idx = molecule.free_indices[displacement_index]  # Pouze jedna souřadnice
    molecule.print_atoms()

    transformer = FreezeCoreTransformer()
    transformed_problem_dx = []
    e_nr_dx = [] 
    for ddx in dxs:
        molecule_mdx = copy.deepcopy(mol)
        molecule_mdx.displace_atom_coordinate(atom_idx, coord_idx, ddx)
        problem_mdx = molecule_mdx.get_pyscf_driver(unit=unit, basis=basis)
        e_nr_dx.append(problem_mdx.nuclear_repulsion_energy)
        main_op_mdx = problem_mdx.hamiltonian.second_q_op()
        molecule_mdx.print_atoms()
        
        transformed_problem_dx.append(transformer.transform(problem_mdx))
        
    
    transformed_problem = transformer.transform(problem)
    
    # GET OPERATORS
    #############################################################################

    num_particles = (problem.num_alpha,
                    problem.num_beta)

    num_spatial_orbitals = problem.num_spatial_orbitals
    num_spin_orbitals = 2 * problem.num_spatial_orbitals
    print("\n\n")
    print(f'Nuclear repulsion energy: {problem.nuclear_repulsion_energy} Ha')
    print(f'Reference energy: {problem.reference_energy} Ha')
    print(f'Number of spin orbitals: {problem.num_spin_orbitals}')
    print(f'Number of alpha electrons: {problem.num_alpha}')
    print(f'Number of beta electrons: {problem.num_beta}')
    print("\n\n")
    #############################################################################
    
    #############################################################################
    
    print(f'Nuclear repulsion energy: {transformed_problem.nuclear_repulsion_energy} Ha')
    print(f'Reference energy: {transformed_problem.reference_energy} Ha')
    print(f'Number of spin orbitals: {transformed_problem.num_spin_orbitals}')
    print(f'Number of alpha electrons: {transformed_problem.num_alpha}')
    print(f'Number of beta electrons: {transformed_problem.num_beta}')
    #############################################################################
    
    # MAPPERS
    #############################################################################
    mapper = None
    if mapper_type == 'JordanWigner':
        mapper = JordanWignerMapper()
    elif mapper_type == 'ParityMapper':
        if qubit_reduction == False:
            mapper = ParityMapper(num_particles=transformed_problem.num_particles)
        else:
            mapper = ParityMapper(num_particles=transformed_problem.num_particles,two_qubit_reduction=True)
    elif mapper_type == 'BravyiKitaev':
        mapper = BravyiKitaevMapper()
    elif mappper_type == 'Superfast':
        mapper = BravyiKitaevSuperFastMapper(num_particles=transformed_problem.num_particles)

    # JordanWigner - https://web.archive.org/web/20191103083720/http://michaelnielsen.org/blog/archive/notes/fermions_and_jordan_wigner.pdf
    # Parity - https://arxiv.org/pdf/1701.08213
    # BravyiKitaev - https://www.sciencedirect.com/science/article/abs/pii/S0003491602962548
    # Superfast - https://arxiv.org/pdf/1712.00446

    # map to qubit operators
    tapered_mapper = transformed_problem.get_tapered_mapper(mapper)

    tapered_mapper_dx = [ t.get_tapered_mapper(mapper) for t in transformed_problem_dx ]

    qubit_op = tapered_mapper.map(transformed_problem.hamiltonian.second_q_op())
 
    qubit_op_dx = [ t.map(transformed_problem_dx[i].hamiltonian.second_q_op()) for i,t in enumerate(tapered_mapper_dx)]
    
    # PREPARE QUANTUM CIRCUIT
    #############################################################################
    
    # BACKEND
    #############################################################################
    
    aer_sim = AerSimulator(method="statevector")
    estimator = Estimator(backend=aer_sim)
    
    ansatz = UCCSD(
        transformed_problem.num_spatial_orbitals,
        transformed_problem.num_particles,
        tapered_mapper,
        initial_state=HartreeFock(
            transformed_problem.num_spatial_orbitals,
            transformed_problem.num_particles,
            tapered_mapper,
        ),
    )

    vqe_solver = VQE(estimator, ansatz=ansatz, optimizer=opt)
    

    # SETUP INITIAL POINT
    #############################################################################
    
    vqe_solver.initial_point = np.zeros(ansatz.num_parameters)
    
    initial_point = HFInitialPoint()
    initial_point.ansatz = ansatz
    initial_point.problem = transformed_problem
    vqe_solver.initial_point = initial_point.to_numpy_array()
    
    #############################################################################
    
    solver = GroundStateEigensolver(tapered_mapper, vqe_solver)
    op, _ = solver.get_qubit_operators(transformed_problem)
    print("\n\n")
    print(f'Number of qubits of main op: {op.num_qubits}, number of paulis: {len(op.paulis)}')

    auxops = { f'aux{dxs[i]}': solver.get_qubit_operators(t)[0] for i,t in enumerate(transformed_problem_dx) }
    
    result_cs = []
    if solver.supports_aux_operators():
        result_cs = solver.solve(transformed_problem, aux_operators=auxops)
    print(result_cs)
    #result_cs = solver.solve(transformed_problem)
    #result_cs2 = solver1.solve(transformed_problem_mdx)
    #result_cs3 = solver2.solve(transformed_problem_pdx)
        
    ###############################################################################
    e = result_cs.eigenvalues + e_nr
    
    #e_pdx = result_cs2.eigenvalues + e_nr_pdx
    #e_mdx = result_cs3.eigenvalues + e_nr_mdx

    e_dx = [ float(result_cs.aux_operators_evaluated[0].get(f'aux{dxs[i]}')) + e_nr_dx[i] for i in range(len(dxs)) ]
    
    return e, e_dx


In [ ]:
# Settings
###################################
basis = 'sto-3g'
unit = DistanceUnit.ANGSTROM
qubit_reduction = False
map_type = "JordanWigner"
shots = 1000
#optimizer = SLSQP(maxiter=10,ftol=1e-9))
#optimizer = SPSA(maxiter=1000)
optimizer = SPSA(maxiter=2000)
ini_params = [ 0, 0, 0]
ddx = 1e-5  # displacement for numerical gradient
###################################

In [ ]:
template_path = "lib/molecule_templates/H2.txt"
config_path = "lib/molecule_configs/H2_1.txt"

steps = 5
dx = 0.05


In [ ]:
mol = generate_molecule_grid(template_path, config_path, 1, 0)[0]
e, e_dx = compute_energy_and_grad(mol,0,shots, qubit_reduction, map_type, parameters=None, opt=optimizer)

In [ ]:
#1000 iter
e, e_dx

In [ ]:
e, e_dx

In [ ]:
c=np.polyfit(dxs,e_dx,2)

In [ ]:

import matplotlib.pyplot as plt


plt.scatter(dxs,e_dx)
plt.scatter([0],[e],color='red')
plt.plot(dxs,np.polyval(c,dxs))
plt.show()